In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [ ]:
url1 = "https://www.bellanaijastyle.com/imad-eduso-lagos-fw-2024/"

response1 = requests.get(url1)
soup1 = BeautifulSoup(response1.content, 'html.parser')

images1 = []
for img1 in soup1.find_all('img1'):
    src = img1.get('src')
    if src and 'imad' in src:
        images1.append(src)
print(f"Found {len(images1)} images from Imad Eduso collection.")

In [ ]:
os.makedirs('imad_eduso_images', exist_ok=True)

for i, img_url in enumerate(images1):
    img_data = requests.get(img_url).content
    with open(os.path.join('imad_eduso_images', f'imad_eduso_{i+1}.jpg'), 'wb') as f:
        f.write(img_data)

In [ ]:
desc = []
for p in soup1.find_all('p', class_='description'):
    text = p.get_text(strip=True)
    if 'Imad Eduso' in text:
        desc.append(text)
    
print(desc[:10])